<a href="https://colab.research.google.com/github/VishnuBhaarath/Computervisionprojects/blob/master/Imageclassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Extracting images from the zip file
from zipfile import ZipFile
filename= "marvel.zip"
with ZipFile(filename,'r')as zip:
  zip.extractall()
  print('extracted successfully')

extracted successfully


In [0]:
#Importing 
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt
try:
 
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
base_dir = os.path.join(os.path.dirname(filename), 'marvel')

In [0]:
# Accessing the images and setting 0.7 of images for training and the rest for testing
classes=['Black Widow','Captain America','Hulk','Iron Man','Thor']
for m in classes:
  img_path = os.path.join(base_dir, m)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(m, len(images)))
  num_train = int(round(len(images)*0.7))
  train, val = images[:num_train], images[num_train:]

# Creating seperate directories for training data
  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', m)):
      os.makedirs(os.path.join(base_dir, 'train', m))
    shutil.move(t, os.path.join(base_dir, 'train', m))
# Creating seperate directories for validating data
  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', m)):
      os.makedirs(os.path.join(base_dir, 'val', m))
    shutil.move(v, os.path.join(base_dir, 'val', m))

Black Widow: 0 Images
Captain America: 0 Images
Hulk: 0 Images
Iron Man: 0 Images
Thor: 0 Images


In [0]:
# Setting train directory and validate directory
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')

In [0]:
# Setting batch size and a constant image shape
batch_size = 130
IMG_SHAPE = 150 
# Rescaling the images so all the values lie between 0 and 1 and applying horizontal flip and training the data
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_data_gen = image_gen.flow_from_directory(
                                                batch_size=batch_size,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE)
                                                )
# Rescaling the images so all the values lie between 0 and 1 and rotating and training the data
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

train_data_gen = image_gen.flow_from_directory(batch_size=batch_size,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE, IMG_SHAPE))
#Rescaling and zooming the data
image_gen_train = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=45,
                    width_shift_range=.15,
                    height_shift_range=.15,
                    horizontal_flip=True,
                    zoom_range=0.5
                    )


train_data_gen = image_gen_train.flow_from_directory(
                                                batch_size=batch_size,
                                                directory=train_dir,
                                                shuffle=True,
                                                target_size=(IMG_SHAPE,IMG_SHAPE),
                                                class_mode='sparse'
                                                )

Found 105 images belonging to 5 classes.
Found 105 images belonging to 5 classes.
Found 105 images belonging to 5 classes.


In [0]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=batch_size,
                                                 directory=val_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='sparse')
model = Sequential()

model.add(Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_SHAPE,IMG_SHAPE, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, 3, padding='same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
# Adding dropout to turn down some neurons
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))

model.add(Dropout(0.2))
model.add(Dense(5, activation='softmax'))

NameError: ignored

In [0]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
epochs = 120

history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=int(np.ceil(train_data_gen.n / float(batch_size))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(val_data_gen.n / float(batch_size)))
)

Epoch 1/120
1/1 [==============================] - 6s 6s/step - loss: 0.3300 - accuracy: 0.8857 - val_loss: 2.6098 - val_accuracy: 0.3778
Epoch 2/120
1/1 [==============================] - 5s 5s/step - loss: 2.1213 - accuracy: 0.5238 - val_loss: 1.5781 - val_accuracy: 0.6000
Epoch 3/120
1/1 [==============================] - 4s 4s/step - loss: 0.5977 - accuracy: 0.7524 - val_loss: 3.2129 - val_accuracy: 0.3556
Epoch 4/120
1/1 [==============================] - 4s 4s/step - loss: 1.7823 - accuracy: 0.5238 - val_loss: 2.3242 - val_accuracy: 0.4222
Epoch 5/120
1/1 [==============================] - 4s 4s/step - loss: 1.2769 - accuracy: 0.5810 - val_loss: 1.4143 - val_accuracy: 0.6000
Epoch 6/120
1/1 [==============================] - 5s 5s/step - loss: 0.5131 - accuracy: 0.8095 - val_loss: 1.3073 - val_accuracy: 0.5333
Epoch 7/120
1/1 [==============================] - 4s 4s/step - loss: 0.5687 - accuracy: 0.7143 - val_loss: 1.3587 - val_accuracy: 0.5333
Epoch 8/120
1/1 [=================

# PROBLEM STATEMENT : Classifying the images of different maravel characters by using neutral network with maximum accuracy.
# MY APPROACH AND LOGIC: I have downloaded 30 images for captain america,black widow,Thor,Hulk and iron man I am uploading the folder and extracting it. Used tensor flow and other essential libraries for this process, usually the images downloaded are of different sizes and different colours, so I convert them to standard size of 150* 150.I used flatten to convert the pixels to a 1D array. The next task is the colour as the colours of grey images are represented by a 1 D array it is simple, but in this case there are colour images I followed the logic that every colour image is a mixture  of red,blue and green, now each colour is of 2D but we have 3 colurs so we end up to with 3 2D arrays. As now there are three colours the input shape parameter has arguments 150,150,3 ,the first two represents the size while the last one 3 represents the number of colours. I did convolution on the given array with a kernel size of 3*3, Then I am making different directories init to train andvalidate the images in such a way that 70%  of the images goes to testing while the rest goes to validation. I have also done max pooling on the convulated image with a size 2*2 and a stride value of 2. I am creating my own directories for train and validate once it is done, I am doing some image augmentation to reduce the risk of overfitting by image rotation, zooming and horizontal flip,,then I am predicting the output.I have also used dropouts to turn down certain neurons to avoid the problem of overfitting and to predict the result more accurate.
#Tools used: google code laboratory,Google images for downloading images. 
#conclusion : I was not able to complete it on time because I have mid sem till saturday and then there was a bit of problem installing the set ups that is the reason why I shifted to codelab,Now I have done the project, I am getting an accuracy of around 70%, the accuracy could still go  up if I have used more images.
#Acknowledgements: I started learning it from scratch and I am happy that I was able to reach to such a level in a span of 2 days, I have really worked hard for the past 2 days from understanding the concepts, I am thankful for databyte for providing me with such a wonderful oppurtunity.

# New Section